In [13]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry

from api_key import APIKEY

API_KEY = APIKEY

In [14]:
CO = ['mp-3414', 'mp-542001', 'mp-28845', 'mp-561423', 'mp-14276', 'mp-29590', 'mp-556442', 'mp-27980']
CH4 = ['mp-680722', 'mp-31107', 'mp-1095378', 'mp-23569', 'mp-30003', 'mp-1190864', 'mp-23084', 'mp-15511', 'mp-7128', 'mp-28855', 'mp-561561', 'mp-1079364', 'mp-542115', 'mp-31232', 'mp-755116', 'mp-1091363', 'mp-558910', 'mp-14037', 'mp-29698', 'mp-1095283', 'mp-20805', 'mp-1113861', 'mp-16834', 'mp-778924', 'mp-556120', 'mp-20727', 'mp-14368', 'mp-1819', 'mp-22467', 'mp-753747', 'mp-15750', 'mp-554435', 'mp-1105700', 'mp-1217378', 'mp-680219', 'mp-17637', 'mp-759342', 'mp-23446', 'mp-779156', 'mp-17677', 'mp-685353', 'mp-230', 'mp-31069', 'mp-1111368', 'mp-759307', 'mp-624234', 'mp-10798', 'mp-760188', 'mp-7094']
CHOOH = ['mp-20015', 'mp-14367', 'mp-541114', 'mp-28299', 'mp-562338', 'mp-554944', 'mp-768936', 'mp-29193']


In [15]:
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in CO:
        material_entries = mpr.materials.search(material_ids=[mp_id])

        if material_entries:
            mp_entries.append(material_entries[0])

chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')]
materials_gga = [material + "-GGA" for material in CO]
materials_id = CO


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
from pymatgen.core.composition import Composition

def remove_o_and_h(composition):
    new_composition_elements = []
    for element, count in composition.get_el_amt_dict().items():
        if element not in ['O', 'H']:
            new_composition_elements.append(f"{element}{int(count)}")
    new_composition_str = ' '.join(new_composition_elements)
    return Composition(new_composition_str)

filtered_molecules = [remove_o_and_h(molecule) for molecule in composition_list]

In [17]:
from pymatgen.core.composition import Composition

def calculate_percent_composition(molecule):
    total_atoms = sum(molecule.values())
    return {element.symbol: count / total_atoms for element, count in molecule.items()}

percent_compositions = [calculate_percent_composition(molecule) for molecule in filtered_molecules]

In [18]:
pbx_data = []
pbx_entries = []

total_iterations = len(chemsys_list)  

with MPRester(API_KEY) as mpr:
    for index, (i, x) in enumerate(zip(chemsys_list, percent_compositions)):
        try:
            pourbaix_entries = mpr.get_pourbaix_entries(i)
            pbx_entries.append(pourbaix_entries)
            pbx_diagram = PourbaixDiagram(entries=pourbaix_entries, comp_dict=x)
            pbx_data.append(pbx_diagram)
        except Exception as e:
            pbx_data.append('error')
        
        percent_complete = ((index + 1) / total_iterations) * 100
        print(f'Percent complete: {percent_complete:.2f}%')

Retrieving ThermoDoc documents:   0%|          | 0/385 [00:00<?, ?it/s]

Percent complete: 12.50%


Retrieving ThermoDoc documents:   0%|          | 0/193 [00:00<?, ?it/s]

Percent complete: 25.00%


Retrieving ThermoDoc documents:   0%|          | 0/184 [00:00<?, ?it/s]

...

Percent complete: 75.00%


Retrieving ThermoDoc documents:   0%|          | 0/163 [00:00<?, ?it/s]

Percent complete: 87.50%


Retrieving ThermoDoc documents:   0%|          | 0/260 [00:00<?, ?it/s]

Percent complete: 100.00%


In [19]:
matched_entries = []

for mat_id, entries in zip(materials_gga, pbx_entries):
    matched_entry = next((entry for entry in entries if entry.entry_id == mat_id), None)
    
    if matched_entry:
        matched_entries.append(matched_entry)
    else:
        matched_entries.append(None)

In [27]:
material_stabilities = []

for i, x in zip(pbx_data, matched_entries):
    stable_count = 0
    total_pH_count = 0

    for pH in range(3, 13):
        try:
            stability = i.get_decomposition_energy(x, pH, 0)
            total_pH_count += 1
            if stability < 0.5:
                stable_count += 1
        except Exception as e:
            continue
    
    if total_pH_count > 0:
        stability_percentage = (stable_count / total_pH_count) * 100
    else:
        stability_percentage = 0

    material_stabilities.append({'entry': x, 'stability_percentage': stability_percentage})

for material in material_stabilities:
    print(f"Material: {material['entry']}, Stability Percentage: {material['stability_percentage']:.2f}%")


Material: PourbaixEntry(Ag4 Sb4 F24 with energy = -68.0689, npH = 0, nPhi = 0, nH2O = 0, entry_id = 'mp-3414-GGA'), Stability Percentage: 70.00%
Material: PourbaixEntry(Ce6 F24 with energy = -114.2972, npH = 0, nPhi = 0, nH2O = 0, entry_id = 'mp-542001-GGA'), Stability Percentage: 100.00%
Material: PourbaixEntry(Os4 O12 F8 with energy = -5.2949, npH = -24.0, nPhi = -24.0, nH2O = 12.0, entry_id = 'mp-28845-GGA'), Stability Percentage: 70.00%
Material: PourbaixEntry(Zr2 Tl2 Cd2 F14 with energy = -59.8113, npH = 0, nPhi = 0, nH2O = 0, entry_id = 'mp-561423-GGA'), Stability Percentage: 100.00%
Material: PourbaixEntry(Rh4 Pb8 F28 with energy = -85.9558, npH = 0, nPhi = 0, nH2O = 0, entry_id = 'mp-14276-GGA'), Stability Percentage: 90.00%
Material: PourbaixEntry(Sn4 O2 F10 with energy = -33.5105, npH = -4.0, nPhi = -4.0, nH2O = 2.0, entry_id = 'mp-29590-GGA'), Stability Percentage: 80.00%
Material: PourbaixEntry(Tc8 O20 F16 with energy = -30.5478, npH = -40.0, nPhi = -40.0, nH2O = 20.0, entr